In [1]:
import mpqp
print(mpqp.__file__)

/Users/yohannedde/Documents/Epita/Quantum/mpqp/Shor/repo-mpqp/mpqp/__init__.py


In [2]:
from mpqp import QCircuit, Barrier
from mpqp.gates import H, X, Y, Z, CNOT, CZ, Id, CRk, SWAP, T
from mpqp.measures import BasisMeasure, I, X as Pauli_X, Y as Pauli_Y, Z as Pauli_Z, PauliString
from mpqp.execution import run, IBMDevice, ATOSDevice, AWSDevice
from mpqp.noise import Depolarizing
from mpqp.core.languages import Language

from math import floor

In [3]:
from mpqp.execution.algorithms.shor.quantum.qft import QFT

print(QFT(range(2, 8)))

                                                          ░               »
q_0: ─────────────────────────────────────────────────────░───────────────»
                                                          ░               »
q_1: ─────────────────────────────────────────────────────░───────────────»
     ┌───┐                                                ░               »
q_2: ┤ H ├─■────────■────────■────────■─────────■─────────░───────────────»
     └───┘ │P(π/2)  │        │        │         │         ░ ┌───┐         »
q_3: ──────■────────┼────────┼────────┼─────────┼─────────░─┤ H ├─■───────»
                    │P(π/4)  │        │         │         ░ └───┘ │P(π/4) »
q_4: ───────────────■────────┼────────┼─────────┼─────────░───────■───────»
                             │P(π/8)  │         │         ░               »
q_5: ────────────────────────■────────┼─────────┼─────────░───────────────»
                                      │P(π/16)  │         ░               »
q_6: ───────

In [4]:
print(QFT(range(5)))

     ┌───┐                                      ░                        »
q_0: ┤ H ├─■────────■────────■────────■─────────░────────────────────────»
     └───┘ │P(π/2)  │        │        │         ░ ┌───┐                  »
q_1: ──────■────────┼────────┼────────┼─────────░─┤ H ├─■────────■───────»
                    │P(π/4)  │        │         ░ └───┘ │P(π/4)  │       »
q_2: ───────────────■────────┼────────┼─────────░───────■────────┼───────»
                             │P(π/8)  │         ░                │P(π/8) »
q_3: ────────────────────────■────────┼─────────░────────────────■───────»
                                      │P(π/16)  ░                        »
q_4: ─────────────────────────────────■─────────░────────────────────────»
                                                ░                        »
«                ░                          ░                 ░       ░       
«q_0: ───────────░──────────────────────────░─────────────────░───────░──X────
«                

In [5]:
#QFT on 1 qubit <=> H
circuit = QFT(range(1))
print(run(circuit, IBMDevice.AER_SIMULATOR))

#QFT on 2^2 qubits <=> H x H
circuit = QFT(range(2**2))
print(run(circuit, IBMDevice.AER_SIMULATOR))

#QFT on 2^3 qubits <=> H x H x H
circuit = QFT(range(2**3))
print(run(circuit, IBMDevice.AER_SIMULATOR))

#QFT then QFT-1 <=> identity
circuit = QFT(range(1)) + QFT(range(1)).inverse()
print(run(circuit, IBMDevice.AER_SIMULATOR))

Result: IBMDevice, AER_SIMULATOR
  State vector: [0.70711, 0.70711]
  Probabilities: [0.5, 0.5]
  Number of qubits: 1
Result: IBMDevice, AER_SIMULATOR
  State vector: [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25]
  Probabilities: [0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625]
  Number of qubits: 4
Result: IBMDevice, AER_SIMULATOR
  State vector: [0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625

In [6]:
from mpqp.gates import Gate, ControlledGate, CustomGate, UnitaryMatrix
from mpqp.measures import BasisMeasure

import numpy as np

# n: the number of qubits
# U should be operate on the target 'target'

def control_U(U: Gate, control: int, target: int) -> Gate:
  assert target > control
  n = target  - control + 1
  In = np.eye(2**(n - 1))
  In_1 = np.eye(2**(n - 2))
  On = np.zeros((2**(n - 1), 2**(n - 1)))
  U_matrix = U.to_matrix()

  Control_U_matrix = np.block([
    [In, On],
    [On, np.kron(In_1, U_matrix)]
    ])

  targets = [i for i in range(control, target + 1)]

  return CustomGate(UnitaryMatrix(Control_U_matrix), targets, 'Control-' + U.label)

print(control_U(X(3), 1, 3))

def qpe(eigenstate, U: Gate, n: int):
  #Prepare state
  circuit1 = QCircuit.initializer(eigenstate)
  circuit1.add(Barrier())
  circuit2 = QCircuit(n)
  circuit_final = circuit2 @ circuit1 #Eigenstate is the last qubit
  for i in range(n):
    circuit_final.add(H(i))
  for i in range(n):
    circuit_final.add(control_U(U(n), n - i - 1, n).power(2**i))
  circuit_final.add(Barrier())
  #QFT not QFT inverse (see MAGNIEZ Lecture Notes)
  circuit_final += QFT(range(n))
  circuit_final.add(BasisMeasure(list(range(n)), shots=10000))
  return circuit_final

                   
q_0: ──────────────
     ┌────────────┐
q_1: ┤2           ├
     │            │
q_2: ┤1 Control-X ├
     │            │
q_3: ┤0           ├
     └────────────┘


In [7]:
from mpqp.execution import Result

# Retreive the state with the higher probability
def getBestGuess(res: Result) -> int:
  bestGuess = 0
  proba = 0
  for i in range(len(res.samples)):
    if (res.samples[i].probability > proba):
      bestGuess = res.samples[i].index
      proba = res.samples[i].probability
  return bestGuess

In [8]:
# Recursive function of "toBinaryRepresentation"
def toBinaryRepresentationIter(n: int, res: list[int]):
  if (n == 1):
    return [1] + res
  return toBinaryRepresentationIter(n // 2, [n % 2] + res)

# Convert an integer into this binary representation
def toBinaryRepresentation(n: int):
  return toBinaryRepresentationIter(n, [])

assert toBinaryRepresentation(3) == [1, 1]
assert toBinaryRepresentation(2) == [1, 0]
assert toBinaryRepresentation(8) == [1, 0, 0, 0]
assert toBinaryRepresentation(9) == [1, 0, 0, 1]

def toPhase(n: int, res: Result):
  bestGuess = getBestGuess(res)
  binary_representation = toBinaryRepresentation(bestGuess)
  sum = 0
  for j in range(n):
    sum += binary_representation[n - j - 1] * 2**j
  return sum / (2**n)

from math import pi
from cmath import exp

def toEigenValue(n: int, res: Result):
  return exp(2 * pi * 1.j * toPhase(n, res))


In [9]:
#Test Control-X

#Control-X with control qubit 0
circuit = QCircuit([control_U(X(0), 0, 3), BasisMeasure([0, 1, 2, 3])])
print(circuit)
print(run(circuit, IBMDevice.AER_SIMULATOR))

#Control-X with control qubit 1
circuit = QCircuit([X(0), control_U(X(10000000000000000000000), 0, 3), BasisMeasure([0, 1, 2, 3])])
print(circuit)
print(run(circuit, IBMDevice.AER_SIMULATOR))

#Inverse of Control-X is Control-X
circuit = QCircuit([X(0), control_U(X(10000), 0, 3), BasisMeasure([0, 1, 2, 3])])
print(circuit)
print(run(circuit, IBMDevice.AER_SIMULATOR))

#Control-H with control qubit 1
circuit = QCircuit([X(0), control_U(H(10000000000000000000000), 0, 3), BasisMeasure([0, 1, 2, 3])])
print(circuit)
print(run(circuit, IBMDevice.AER_SIMULATOR))

     ┌────────────┐┌─┐         
q_0: ┤3           ├┤M├─────────
     │            │└╥┘┌─┐      
q_1: ┤2           ├─╫─┤M├──────
     │  Control-X │ ║ └╥┘┌─┐   
q_2: ┤1           ├─╫──╫─┤M├───
     │            │ ║  ║ └╥┘┌─┐
q_3: ┤0           ├─╫──╫──╫─┤M├
     └────────────┘ ║  ║  ║ └╥┘
c: 4/═══════════════╩══╩══╩══╩═
                    0  1  2  3 
Result: IBMDevice, AER_SIMULATOR
  Counts: [1024, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  Probabilities: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  Samples:
    State: 0000, Index: 0, Count: 1024, Probability: 1
  Error: None
     ┌───┐┌────────────┐┌─┐         
q_0: ┤ X ├┤3           ├┤M├─────────
     └───┘│            │└╥┘┌─┐      
q_1: ─────┤2           ├─╫─┤M├──────
          │  Control-X │ ║ └╥┘┌─┐   
q_2: ─────┤1           ├─╫──╫─┤M├───
          │            │ ║  ║ └╥┘┌─┐
q_3: ─────┤0           ├─╫──╫──╫─┤M├
          └────────────┘ ║  ║  ║ └╥┘
c: 4/════════════════════╩══╩══╩══╩═
                         0  1  2  3 


In [10]:
from math import sqrt

eigenstate = np.array([0, 1])
U = X
n = 5

print(qpe(eigenstate, U, n))

                  ░ ┌───┐                                              »
q_0: ─────────────░─┤ H ├──────────────────────────────────────────────»
                  ░ ├───┤                                              »
q_1: ─────────────░─┤ H ├──────────────────────────────────────────────»
                  ░ ├───┤                              ┌──────────────┐»
q_2: ─────────────░─┤ H ├──────────────────────────────┤3             ├»
                  ░ ├───┤              ┌──────────────┐│              │»
q_3: ─────────────░─┤ H ├──────────────┤2             ├┤2             ├»
                  ░ ├───┤┌────────────┐│              ││  Control-X^4 │»
q_4: ─────────────░─┤ H ├┤1           ├┤1 Control-X^2 ├┤1             ├»
     ┌──────────┐ ░ └───┘│  Control-X ││              ││              │»
q_5: ┤ U(π,0,0) ├─░──────┤0           ├┤0             ├┤0             ├»
     └──────────┘ ░      └────────────┘└──────────────┘└──────────────┘»
c: 5/══════════════════════════════════════════════

In [11]:
#Test X and n = 1
eigenstate = np.array([0, 1])
U = Z
n = 2 # Because pi can be represented with 2 bits 11 (approximated to 3)

print(qpe(eigenstate, U, n))

res = run(qpe(eigenstate, U, n), IBMDevice.AER_SIMULATOR)
print(toEigenValue(n, res))
assert toPhase(n, res) == 1/2 #=> 1/2

                  ░ ┌───┐              ┌──────────────┐ ░ ┌───┐          ░ »
q_0: ─────────────░─┤ H ├──────────────┤2             ├─░─┤ H ├─■────────░─»
                  ░ ├───┤┌────────────┐│              │ ░ └───┘ │P(π/2)  ░ »
q_1: ─────────────░─┤ H ├┤1           ├┤1 Control-Z^2 ├─░───────■────────░─»
     ┌──────────┐ ░ └───┘│  Control-Z ││              │ ░                ░ »
q_2: ┤ U(π,0,0) ├─░──────┤0           ├┤0             ├─░────────────────░─»
     └──────────┘ ░      └────────────┘└──────────────┘ ░                ░ »
c: 2/══════════════════════════════════════════════════════════════════════»
                                                                           »
«           ░    ┌─┐   
«q_0: ──────░──X─┤M├───
«     ┌───┐ ░  │ └╥┘┌─┐
«q_1: ┤ H ├─░──X──╫─┤M├
«     └───┘ ░     ║ └╥┘
«q_2: ──────░─────╫──╫─
«           ░     ║  ║ 
«c: 2/════════════╩══╩═
«                 0  1 
(-1+1.2246467991473532e-16j)


In [12]:
#Y: Y[-i> = - [-i>

eigenstate = np.array([1, -1.j]) / sqrt(2) #[-i> = ([0> - j[1>) / sqrt(2) = [1/sqrt(2), -j/sqrt(2)]
U = Y
n = 2 # Because pi can be represented with 2 bits 11 (approximated to 3)

print(qpe(eigenstate, U, n))

print(run(qpe(eigenstate, U, n), IBMDevice.AER_SIMULATOR)) #=> 1/2

                         ░ ┌───┐              ┌──────────────┐ ░ ┌───┐         »
q_0: ────────────────────░─┤ H ├──────────────┤2             ├─░─┤ H ├─■───────»
                         ░ ├───┤┌────────────┐│              │ ░ └───┘ │P(π/2) »
q_1: ────────────────────░─┤ H ├┤1           ├┤1 Control-Y^2 ├─░───────■───────»
     ┌─────────────────┐ ░ └───┘│  Control-Y ││              │ ░               »
q_2: ┤ U(π/2,-π/2,π/2) ├─░──────┤0           ├┤0             ├─░───────────────»
     └─────────────────┘ ░      └────────────┘└──────────────┘ ░               »
c: 2/══════════════════════════════════════════════════════════════════════════»
                                                                               »
«      ░       ░    ┌─┐   
«q_0: ─░───────░──X─┤M├───
«      ░ ┌───┐ ░  │ └╥┘┌─┐
«q_1: ─░─┤ H ├─░──X──╫─┤M├
«      ░ └───┘ ░     ║ └╥┘
«q_2: ─░───────░─────╫──╫─
«      ░       ░     ║  ║ 
«c: 2/═══════════════╩══╩═
«                    0  1 
Result: IBMDevice, AER_SIMUL

In [13]:
# Z[0> = exp(0.j) * [0>
eigenstate = np.array([1, 0])
U = Z
n = 5

print(run(qpe(eigenstate, U, n), IBMDevice.AER_SIMULATOR))

# Z[1> = exp(pi.j) * [1>
eigenstate = np.array([0, 1])
U = Z
n = 6

print(run(qpe(eigenstate, U, n), IBMDevice.AER_SIMULATOR))

Result: IBMDevice, AER_SIMULATOR
  Counts: [10000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  Probabilities: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  Samples:
    State: 000000, Index: 0, Count: 10000, Probability: 1
  Error: None
Result: IBMDevice, AER_SIMULATOR
  Counts: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  Probabilities: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  Samples:
    State: 0100000, Index: 32, Count: 10000, Probability: 1
  Error: None


In [14]:
from math import sqrt
circuit = QCircuit.initializer(np.array([1, 1])/sqrt(2))

print(circuit)

   ┌────────────┐
q: ┤ U(π/2,0,0) ├
   └────────────┘


In [15]:
circuit = QCircuit.initializer(np.array([0, 1]))

print(circuit)

   ┌──────────┐
q: ┤ U(π,0,0) ├
   └──────────┘


In [16]:
circuit = QCircuit.initializer(np.array([sqrt(0.2), sqrt(0.8)]))

print(circuit)

   ┌───────────────┐
q: ┤ U(2.2143,0,0) ├
   └───────────────┘


In [17]:
circuit = QCircuit.initializer(np.array([1, 1, 1, 1]) / sqrt(2))

print(circuit)

        ┌────────────┐                            
q_0: ───┤ U(π/2,0,0) ├────■───────────────────────
     ┌──┴────────────┴─┐┌─┴─┐┌───────────────────┐
q_1: ┤ U(π/2,-2π,-π/2) ├┤ X ├┤ U(0,-1.543,1.543) ├
     └─────────────────┘└───┘└───────────────────┘


In [18]:
swap_gate = SWAP(0,1)
from mpqp.gates import X
print((X(1).power(2)).to_matrix())

[[1. 0.]
 [0. 1.]]


In [19]:
# Euclide algorithm
def euclide(a, b):
  while b != 0:
    tmp = b
    b = a % b
    a = tmp
  return a

assert(euclide(21, 15) == 3)
assert(euclide(210, 63) == 21)
assert(euclide(107, 23) == 1)

In [20]:
import numpy as np

# Choose a random integer 'a' which is a factor of 'N'
def getFactor(N: int) -> int:
  a = np.random.randint(2, N)
  while euclide(a, N) == 1:
    a = np.random.randint(2, N)
  return a

a = getFactor(15)

In [21]:
# using continued fractions estimate the period r
def continued_fraction(p, q):
    a = []
    while q:
        a.append(p // q)
        p, q = q, p % q
    return a

assert(continued_fraction(9, 4) == [2, 4])
assert(continued_fraction(30, 13) == [2, 3, 4])
assert(continued_fraction(2, 17) == [0, 8, 2])

In [22]:
def verify_period(a, r, N):
  return (a**r) % N == 1

verify_period(a, 2, 15)

False

In [23]:
# Fast exponantiation modulo m
def fast_expo(a, n, m=None):
  if n == 0:
    return 1
  b = (a*a)
  if m != None:
    b = b % m
  if n % 2 == 0:
    return fast_expo(b, n // 2, m)
  return a * fast_expo(b, n // 2, m)

assert(fast_expo(2, 8, 10) == 6)
assert(fast_expo(3, 5, 10) == 3)

In [24]:
def compute_factors(a, r, N):
  if r % 2 == 0 and fast_expo(a, r // 2, N) != -1 % N:
    p = fast_expo(a, r // 2)
    return euclide(p-1, N), euclide(p+1, N)

In [25]:
from mpqp.execution.algorithms.shor.quantum.modularExponentiationGate import ME, CME

print(ME(2, 11, 0, list(range(2, 6))))
print(CME(2, 11, 0, [0],list(range(2, 6))))

                       
q_0: ──────────────────
                       
q_1: ──────────────────
     ┌────────────────┐
q_2: ┤3               ├
     │                │
q_3: ┤2               ├
     │  a^(2^0) mod N │
q_4: ┤1               ├
     │                │
q_5: ┤0               ├
     └────────────────┘
                 
q_0: ─────■──────
          │      
q_1: ─────┼──────
     ┌────┴─────┐
q_2: ┤0         ├
     │          │
q_3: ┤1         ├
     │  Unitary │
q_4: ┤2         ├
     │          │
q_5: ┤3         ├
     └──────────┘


In [26]:
from math import log

"""
  a: choosen number, inversible in Zn.
  k: repitition number. Represents the exponent of the gate in the shor's qpe.
  n: number of bits of the second register. Also represents the number of target qubits
    in the shor's qpe
  N: number to be factorized
"""

def kth_power_a_mod_N_oracle(a, k, N, targets):
  #Initialise n
  n = floor(log(N - 1, 2)) + 1
  assert len(targets) == n

  #Initialise b = a^(2^k) mod N
  b = fast_expo(a, 2**k, N)

  #Initialise the matrix with zeros and dimension 2^n * 2^n
  d = fast_expo(2, n)
  U = np.zeros((d, d))

  #By definition, U[j> = [b * j mod N> for j in {0, ..., N - 1}
  for j in range(N):
    U[(b * j) % N][j] = 1

  #Then the operator is completed to be applied on all integers from 0 to 2 ** n
  #Nothing is done for j in {N, ..., 2**n}
  for j in range(N, 2**n):
    U[j][j] = 1

  #U is necessarily unitary as b is inversible in Zn
  return CustomGate(UnitaryMatrix(U), targets, 'a^(2^' + str(k) + ') mod N')


In [27]:
kth_power_a_mod_N_oracle(2, 1, 3, [0, 1])

CustomGate(UnitaryMatrix(np.array([[1., 0., 0., 0.], [0., 1., 0., 0.], [0., 0., 1., 0.], [0., 0., 0., 1.]])), [0, 1], "a^(2^1) mod N")

In [28]:
"""
Test: a = 7, N = 15, k = 0.
Circuit with gate kth_power_a_mod_N_oracle(7, 0, 15)
and initial state 1000 = 8 should return final state 1011 = 11
"""


def Test():
  a = 7
  k = 0
  N = 15
  n = floor(log(N-1, 2)) + 1
  circuit = QCircuit(n)
  circuit.add(X(0))
  circuit.add(kth_power_a_mod_N_oracle(a, k, N, list(range(4))))
  circuit.add(BasisMeasure(list(range(n)), shots=1000))
  return getBestGuess(run(circuit, IBMDevice.AER_SIMULATOR))

assert Test() == 11


In [29]:
# circuit = QCircuit(2)
# circuit.add(X(0))
# circuit.add(CustomControlledGate([0], X(1)))
# circuit.add(BasisMeasure([0, 1], shots=1000))
# print(circuit)
# print(run(circuit, IBMDevice.AER_SIMULATOR))
from mpqp.core.instruction.gates.custom_controlled_gate import CustomControlledGate

circuit = QCircuit(2)
circuit.add(X(0))
circuit.add(CustomControlledGate([0], CustomGate(
    UnitaryMatrix(X(1).to_matrix()),
    [1])))
circuit.add(BasisMeasure([0, 1], shots=1000))
print(circuit)


print(run(circuit, IBMDevice.AER_SIMULATOR))


     ┌───┐           ┌─┐   
q_0: ┤ X ├─────■─────┤M├───
     └───┘┌────┴────┐└╥┘┌─┐
q_1: ─────┤ Unitary ├─╫─┤M├
          └─────────┘ ║ └╥┘
c: 2/═════════════════╩══╩═
                      0  1 
Result: IBMDevice, AER_SIMULATOR
  Counts: [0, 0, 0, 1000]
  Probabilities: [0, 0, 0, 1]
  Samples:
    State: 11, Index: 3, Count: 1000, Probability: 1
  Error: None


In [30]:
def control_U(U: Gate, control: int, target: int) -> Gate:
  assert target > control
  #assert U.to_matrix().shape == (2, 2)
  U_matrix = U.to_matrix()

  r = floor(log(U_matrix.shape[0], 2))
  n = target  - control + 1
  In = np.eye(2**(n - 1))
  In_1 = np.eye(2**(n - 1 - r))
  On = np.zeros((2**(n - 1), 2**(n - 1)))

  Control_U_matrix = np.block([
    [In, On],
    [On, np.kron(In_1, U_matrix)]
    ])

  targets = [i for i in range(control, target + 1)]

  return CustomGate(UnitaryMatrix(Control_U_matrix), targets, 'Control-' + U.label)

In [31]:

circuit = QCircuit(7)
circuit.add(X(0))

my_gate = CustomGate(
    UnitaryMatrix(X(1).to_matrix()), 
    [1],
    "X"
)

circuit.add(control_U(my_gate, 0, 4))
circuit.add(control_U(my_gate, 0, 5))
circuit.add(control_U(my_gate, 0, 6))

circuit.add(BasisMeasure([0, 1, 2, 3, 4, 5, 6], shots=1000))

print(circuit)
print(run(circuit, IBMDevice.AER_SIMULATOR))

     ┌───┐┌────────────┐┌────────────┐┌────────────┐┌─┐                  
q_0: ┤ X ├┤4           ├┤5           ├┤6           ├┤M├──────────────────
     └───┘│            ││            ││            │└╥┘┌─┐               
q_1: ─────┤3           ├┤4           ├┤5           ├─╫─┤M├───────────────
          │            ││            ││            │ ║ └╥┘┌─┐            
q_2: ─────┤2 Control-X ├┤3           ├┤4           ├─╫──╫─┤M├────────────
          │            ││  Control-X ││            │ ║  ║ └╥┘┌─┐         
q_3: ─────┤1           ├┤2           ├┤3 Control-X ├─╫──╫──╫─┤M├─────────
          │            ││            ││            │ ║  ║  ║ └╥┘┌─┐      
q_4: ─────┤0           ├┤1           ├┤2           ├─╫──╫──╫──╫─┤M├──────
          └────────────┘│            ││            │ ║  ║  ║  ║ └╥┘┌─┐   
q_5: ───────────────────┤0           ├┤1           ├─╫──╫──╫──╫──╫─┤M├───
                        └────────────┘│            │ ║  ║  ║  ║  ║ └╥┘┌─┐
q_6: ─────────────────────────────────

In [38]:
from math import floor, log
from mpqp.execution.algorithms.shor.quantum.modularExponentiationGate import CME
from mpqp.execution.algorithms.shor.quantum.qft import QFT

def qpe(a: int, N: int):
  #Initialise the number of target qubits
  n = floor(log(N - 1, 2)) + 1

  #Circuit with 2*n controls and n targets
  circuit = QCircuit(2 * n + n)

  #Initialise the state of the targets
  circuit.add(X(2 * n))

  circuit.add(Barrier())
  
  for k in range(2 * n):
    circuit.add(H(k))
    circuit.add(CME(a, N, k, [k], list(range(2 * n, 3 * n))))
  
  circuit.add(Barrier())

  #QFT not QFT inverse (see MAGNIEZ Lecture Notes)
  circuit += QFT(range(2 * n)).inverse()

  circuit.add(BasisMeasure(list(range(2 * n)), shots=10000))

  return circuit

N = 11
a = 2
print(qpe(a, N))

            ░ ┌───┐                                                            »
 q_0: ──────░─┤ H ├─────■──────────────────────────────────────────────────────»
            ░ ├───┤     │                                                      »
 q_1: ──────░─┤ H ├─────┼───────────■──────────────────────────────────────────»
            ░ ├───┤     │           │                                          »
 q_2: ──────░─┤ H ├─────┼───────────┼───────────■──────────────────────────────»
            ░ ├───┤     │           │           │                              »
 q_3: ──────░─┤ H ├─────┼───────────┼───────────┼───────────■──────────────────»
            ░ ├───┤     │           │           │           │                  »
 q_4: ──────░─┤ H ├─────┼───────────┼───────────┼───────────┼───────────■──────»
            ░ ├───┤     │           │           │           │           │      »
 q_5: ──────░─┤ H ├─────┼───────────┼───────────┼───────────┼───────────┼──────»
            ░ ├───┤     │   

In [39]:
from mpqp.execution.algorithms.shor.quantum.shorCircuit import ShorCircuit
print(ShorCircuit(2, 11))

ModuleNotFoundError: No module named 'mpqp.execution.algorithms.shor.modularExponentiationGate'

In [ ]:
print(run(qpe(2, 11), IBMDevice.AER_SIMULATOR))

##

Result: IBMDevice, AER_SIMULATOR
  Counts: [990, 1029, 0, 2, 1, 3, 3, 4, 0, 2, 5, 1, 0, 1, 3, 1, 4, 5, 2, 3, 34, 28, 14, 17, 1, 5, 3, 2, 15, 36, 19, 15, 0, 0, 19, 24, 2, 3, 7, 18, 10, 15, 188, 138, 11, 6, 74, 73, 4, 2, 21, 18, 26, 26, 25, 34, 24, 25, 199, 241, 31, 32, 121, 127, 1, 2, 2, 3, 26, 43, 11, 25, 1, 1, 10, 14, 10, 25, 13, 21, 11, 21, 8, 8, 115, 120, 68, 54, 6, 6, 18, 19, 35, 44, 30, 33, 1, 5, 15, 17, 8, 18, 8, 19, 14, 21, 184, 179, 20, 21, 131, 142, 12, 9, 13, 9, 81, 83, 61, 54, 13, 7, 72, 56, 40, 30, 43, 36, 1, 2, 4, 8, 0, 1, 5, 4, 4, 5, 36, 47, 0, 0, 33, 25, 3, 6, 8, 7, 62, 54, 20, 32, 9, 3, 21, 14, 48, 58, 51, 32, 7, 2, 25, 26, 4, 4, 20, 16, 13, 19, 170, 153, 7, 9, 61, 58, 4, 4, 14, 10, 27, 33, 16, 24, 11, 10, 76, 76, 14, 15, 21, 34, 1, 2, 0, 1, 13, 20, 3, 7, 1, 1, 27, 19, 8, 8, 17, 19, 38, 20, 22, 17, 237, 238, 107, 113, 12, 8, 31, 17, 129, 150, 90, 67, 2, 2, 10, 10, 5, 13, 13, 13, 9, 10, 81, 98, 9, 5, 41, 57, 11, 11, 8, 7, 119, 99, 59, 66, 5, 7, 22, 8, 52, 52, 26, 23]
  P